# Iteratively Train the Underlying MLP Models in TELL Using Evolving Time Windows

This notebook is the core tool for training the evolving sets of MLP models used in this LDRD. It assumes that the base Total ELectricity Loads (TELL) model has already been installed and the datasets updated to include the most recent data.

In [1]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell


## Set the Directory Structure


In [19]:
# Identify the top-level directory to store the trained MLP models:
ba_data_input_dir =  '/Users/burl878/Documents/Code/code_repos/tell/tell/tell_data/tell_quickstarter_data/outputs/compiled_historical_data/'
model_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/trained_mlp_models/'


## MLP Model Training

The MLP models underpinning TELL use temporal variations in weather to project hourly demand. More information about this approach is in the MLP section of the `tell` [User Guide](https://immm-sfa.github.io/tell/user_guide.html). We include pre-trained models within the `tell` repository. Note that since the `save_model` parameter is set to false by default running these training steps will not overwrite the models included in `tell`. The default settings for the MLP model training steps are included in the `mlp_settings.yml` file included in the data folder of the `tell` repository. By default the MLP models are trained on data from 2016-2018 and evaluated using data from 2019. The time windows for training and evaluating the models can be modified by altering the `start_time`, `end_time`, and `split_datetime` parameters when calling the `tell.train` function. The following statistical values, all computed using the sklearn package, are used to evaluate the MLP models:

| Parameter | Description | Documentation |
| :-: | :- | :-: |
| R2 | Coefficient of determination | [sklearn.metrics.r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) |
| RMS_ABS | Root-mean-squared of the absolute error | [sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) |
| RMS_NORM| The RMS_ABS value divided by the mean | [sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) |
| MAPE| Mean absolute percentage error | [sklearn.metrics.mean_absolute_percentage_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html#sklearn.metrics.mean_absolute_percentage_error) |

In [3]:
# For more information about the training of predictive models you can call the help function:
help(tell.train)


Help on function train in module tell.mlp_train:

train(region: str, data_dir: str, **kwargs)
    Generate predictions for MLP model for a target region from an input CSV file.
    
    :param region:                      Indicating region / balancing authority we want to train and test on.
                                        Must match with string in CSV files.
    :type region:                       str
    
    :param data_dir:                    Full path to the directory that houses the input CSV files.
    :type data_dir:                     str
    
    :param mlp_hidden_layer_sizes:      The ith element represents the number of neurons in the ith hidden layer.
    :type mlp_hidden_layer_sizes:       Optional[int]
    
    :param mlp_max_iter:                Maximum number of iterations. The solver iterates until convergence
                                        (determined by ‘tol’) or this number of iterations. For stochastic solvers
                                     

In [9]:
# Run the MLP training step for a single BA to get a feel of the functionality:
prediction_df, validation_df = tell.train(region = 'CISO',
                                          data_dir = '/Users/burl878/Documents/Code/code_repos/tell/tell/tell_data/tell_quickstarter_data/outputs/compiled_historical_data/',
                                          start_time = '2018-01-01 00:00:00',
                                          end_time = '2020-12-31 23:00:00',
                                          split_datetime = '2019-12-31 23:00:00',
                                          save_model = True,
                                          model_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/trained_mlp_models/M1/')

# View the head of the prediction dataframe that contains the time-series of projected load in the evaluation year:
display(prediction_df.head(10))

# View validation dataframe that contains error statistics for the trained model:
validation_df


,datetime,predictions,ground_truth,region
0,2020-01-01 00:00:00,19558.855040,22497.0,CISO
1,2020-01-01 01:00:00,24234.389987,24297.0,CISO
2,2020-01-01 02:00:00,24867.025706,26918.0,CISO
3,2020-01-01 03:00:00,25423.816113,26849.0,CISO
4,2020-01-01 04:00:00,25082.532154,25968.0,CISO
5,2020-01-01 05:00:00,24106.800709,25044.0,CISO
6,2020-01-01 06:00:00,23192.030982,24185.0,CISO
7,2020-01-01 07:00:00,22227.265649,23305.0,CISO
8,2020-01-01 08:00:00,21233.518247,22467.0,CISO
9,2020-01-01 09:00:00,20275.718493,21810.0,CISO


,BA,RMS_ABS,RMS_NORM,MAPE,R2
0,CISO,1990.201611,0.080216,0.058195,0.859668


In [27]:
# Run the training iteratively across all training windows and BAs:
for model in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']:

    # Set the training and evaluation periods for the specific model training window:
    if model == 'M1':
       start_time_model = '2016-01-01 00:00:00'
       end_time_model = '2018-12-31 23:00:00'
       split_datetime_model = '2017-12-31 23:00:00'
    if model == 'M2':
       start_time_model = '2017-01-01 00:00:00'
       end_time_model = '2019-12-31 23:00:00'
       split_datetime_model = '2018-12-31 23:00:00'
    if model == 'M3':
       start_time_model = '2018-01-01 00:00:00'
       end_time_model = '2020-12-31 23:00:00'
       split_datetime_model = '2019-12-31 23:00:00'
    if model == 'M4':
       start_time_model = '2019-01-01 00:00:00'
       end_time_model = '2021-12-31 23:00:00'
       split_datetime_model = '2020-12-31 23:00:00'
    if model == 'M5':
       start_time_model = '2020-01-01 00:00:00'
       end_time_model = '2022-12-31 23:00:00'
       split_datetime_model = '2021-12-31 23:00:00'
    if model == 'M6':
       start_time_model = '2021-01-01 00:00:00'
       end_time_model = '2023-12-31 23:00:00'
       split_datetime_model = '2022-12-31 23:00:00'

    # Create the model output directory name:
    output_directory = (model_output_directory + model + '/')
  
    # Check to see if the output directory exist and if not then create it:
    if not os.path.exists(output_directory):
       os.makedirs(output_directory)

    # Loop over the eight BAs used in this LDRD analysis:
    for ba in ['AZPS', 'BPAT', 'CISO', 'ERCO', 'FPL', 'ISNE', 'PJM', 'WACM']:
        
        # Run the MLP training for that BA and model training period:
        prediction_df, validation_df = tell.train(region = ba,
                                                  data_dir = ba_data_input_dir,
                                                  start_time = start_time_model,
                                                  end_time = end_time_model,
                                                  split_datetime = split_datetime_model,
                                                  save_model = True,
                                                  model_output_directory = output_directory)

        # Print the model and BA combination to monitor the progress:
        print('Model = ', model, ', BA = ', ba)


Model =  M1 , BA =  AZPS
Model =  M1 , BA =  BPAT
Model =  M1 , BA =  CISO
Model =  M1 , BA =  ERCO
Model =  M1 , BA =  FPL
Model =  M1 , BA =  ISNE
Model =  M1 , BA =  PJM
Model =  M1 , BA =  WACM
Model =  M2 , BA =  AZPS
Model =  M2 , BA =  BPAT
Model =  M2 , BA =  CISO
Model =  M2 , BA =  ERCO
Model =  M2 , BA =  FPL
Model =  M2 , BA =  ISNE
Model =  M2 , BA =  PJM
Model =  M2 , BA =  WACM
Model =  M3 , BA =  AZPS
Model =  M3 , BA =  BPAT
Model =  M3 , BA =  CISO
Model =  M3 , BA =  ERCO
Model =  M3 , BA =  FPL
Model =  M3 , BA =  ISNE
Model =  M3 , BA =  PJM
Model =  M3 , BA =  WACM
Model =  M4 , BA =  AZPS
Model =  M4 , BA =  BPAT
Model =  M4 , BA =  CISO
Model =  M4 , BA =  ERCO
Model =  M4 , BA =  FPL
Model =  M4 , BA =  ISNE
Model =  M4 , BA =  PJM
Model =  M4 , BA =  WACM
Model =  M5 , BA =  AZPS
Model =  M5 , BA =  BPAT
Model =  M5 , BA =  CISO
Model =  M5 , BA =  ERCO
Model =  M5 , BA =  FPL
Model =  M5 , BA =  ISNE
Model =  M5 , BA =  PJM
Model =  M5 , BA =  WACM
Model =  M